# Отчёт по конкурсу "Avito Demand Prediction Challenge"
##### Выполнили:
Женя Глазунов и Ира Пантелеева

In [94]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import TweetTokenizer
import re
from tqdm import tqdm
from model import Model
import numpy as np

Загружаем тестовую и обучающую выборки:

In [96]:
train_data = pd.read_csv('/Users/irene/Downloads/project/train_avito.csv')
test_data = pd.read_csv('/Users/irene/Downloads/project/test_avito.csv')
train_data = train_data[['description', 'title']]
test_data = test_data[['description', 'title']]

In [97]:
train_data.head()

,description,title
0,"Кокон для сна малыша,пользовались меньше месяц...",Кокоби(кокон для сна)
1,"Стойка для одежды, под вешалки. С бутика.",Стойка для Одежды
2,"В хорошем состоянии, домашний кинотеатр с blu ...",Philips bluray
3,Продам кресло от0-25кг,Автокресло
4,Все вопросы по телефону.,"ВАЗ 2110, 2003"


# Препроцессинг

## Токенизация

In [10]:
tknzr = TweetTokenizer()
stops = stopwords.words('russian')

Удаляем адреса:

In [11]:
def remove_sites(text):
    no_sites_text = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?\S',
                           ' ', text, flags=re.MULTILINE)
    no_space_sequences_text = re.sub('  +', ' ', no_sites_text)
    return no_space_sequences_text

Убираем стоп-слова:

In [12]:
def normalize_stop(text):
    text = remove_sites(text)
    lemmas = [morph.parse(token)[0].normal_form for token in tknzr.tokenize(text)]
    return ' '.join(lemmas)

In [19]:
train_data['tokenized_title'] = train_data['title'].apply(normalize_stop)
train_data['description'] = train_data['description'].fillna(value='')
train_data['tokenized_description'] = train_data['description'].apply(normalize_stop)

In [22]:
train_data.head()

,description,title,tokenized_title,tokenized_description
0,"Кокон для сна малыша,пользовались меньше месяц...",Кокоби(кокон для сна),кокоби ( кокон для сон ),"кокон для сон малыш , пользоваться маленький м..."
1,"Стойка для одежды, под вешалки. С бутика.",Стойка для Одежды,стойка для одежда,"стойка для одежда , под вешалка . с бутик ."
2,"В хорошем состоянии, домашний кинотеатр с blu ...",Philips bluray,philips bluray,"в хороший состояние , домашний кинотеатр с blu..."
3,Продам кресло от0-25кг,Автокресло,автокресло,продать кресло от0 - 25кг
4,Все вопросы по телефону.,"ВАЗ 2110, 2003","ваза 2110 , 2003",весь вопрос по телефон .


## Стемминг

In [24]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer('russian')

In [25]:
def normalize_stem(text):
    stems = [stemmer.stem(token) for token in tknzr.tokenize(text)]
    return ' '.join(stems)

In [26]:
train_data['tokenized_stemmed_title'] = train_data['tokenized_title'].apply(normalize_stem)
train_data['tokenized_stemmed_description'] = train_data['tokenized_description'].apply(normalize_stem)
train_data.to_csv('train_data_normalized.csv', encoding='utf-8')

Токенизируем и делаем стемминг на тестовой выборке:

In [28]:
test_data['tokenized_title'] = test_data['title'].apply(normalize_stop)
test_data['description'] = test_data['description'].fillna(value='')
test_data['tokenized_description'] = test_data['description'].apply(normalize_stop)
test_data['tokenized_stemmed_title'] = test_data['tokenized_title'].apply(normalize_stem)
test_data['tokenized_stemmed_description'] = test_data['tokenized_description'].apply(normalize_stem)
test_data.to_csv('test_data_normalized.csv', encoding='utf-8')

## Частеречная разметка c помощью UDPipe (+ типы связи)

In [2]:
model = Model('russian-ud-2.0-170801.udpipe')

In [3]:
def get_chains(text):
    try:
        sentences = model.tokenize(text)
        for s in sentences:
            model.tag(s)
            model.parse(s)
        conllu = model.write(sentences, "conllu")
        rows = conllu.split('\n')
        string_pos = ''
        string_rel_type = ''
        string_token = ''
        for row in rows:
            if '#' not in row and row != '':
                pos = re.search('\t.+?\t.+?\t(.+?)\t', row).group(1)
                rel_type = re.search('\t.+?\t.+?\t.+?\t.+?\t.+?\t.+?\t(.+?)\t', row).group(1)
                token = re.search('\t(.+?)\t', row).group(1)
                string_pos = string_pos + pos + ' '
                string_rel_type = string_rel_type + rel_type + ' '
                string_token = string_token + token + ' '
        return string_token[:-1], string_pos[:-1], string_rel_type[:-1]
    except:
        return '', '', ''

Пример:

In [4]:
get_chains('Ира пришла в вуз. Она ушла в магазин.')

('Ира пришла в вуз . Она ушла в магазин .',
 'NOUN VERB ADP NOUN PUNCT PRON VERB ADP NOUN PUNCT',
 'nsubj root case obl punct nsubj root case obl punct')

In [5]:
train_data = pd.read_csv('train_data_normalized.csv')
train_data = train_data.drop(['Unnamed: 0'], axis=1)

In [10]:
train_data.head()

,description,title,tokenized_title,tokenized_description,tokenized_stemmed_title,tokenized_stemmed_description
0,"Кокон для сна малыша,пользовались меньше месяц...",Кокоби(кокон для сна),кокоби ( кокон для сон ),"кокон для сон малыш , пользоваться маленький м...",кокоб ( кокон для сон ),"кокон для сон малыш , пользова маленьк месяц ...."
1,"Стойка для одежды, под вешалки. С бутика.",Стойка для Одежды,стойка для одежда,"стойка для одежда , под вешалка . с бутик .",стойк для одежд,"стойк для одежд , под вешалк . с бутик ."
2,"В хорошем состоянии, домашний кинотеатр с blu ...",Philips bluray,philips bluray,"в хороший состояние , домашний кинотеатр с blu...",philips blura,"в хорош состоян , домашн кинотеатр с blu ra , ..."
3,Продам кресло от0-25кг,Автокресло,автокресло,продать кресло от0 - 25кг,автокресл,прода кресл от0 - 25кг
4,Все вопросы по телефону.,"ВАЗ 2110, 2003","ваза 2110 , 2003",весь вопрос по телефон .,"ваз 2110 , 2003",ве вопрос по телефон .


In [49]:
res_train = pd.DataFrame(data={'tokens': [], 'pos': [], 'type': []})

Делим выборку на части для отслеживания работы функции:

In [51]:
l = []
k = 0
for i in range(0, 1503424, 500):
    l.append((k, i))
    k = i
l = l[1:]
l.append((1503000, 1503424))

In [98]:
for every in l:
    tok_pos_type_train = train_data[every[0]:every[1]].description.apply(lambda s: pd.Series(get_chains(s)))
    tok_pos_type_train.columns = ['tokens', 'pos', 'type']
    res_train = pd.concat([res_train, tok_pos_type_train])
    #print('Конец обработки на', every)

In [56]:
res_train.to_csv('tok_pos_type_train.csv', encoding='utf-8')

То же самое делаем на трейне:

In [66]:
l = []
k = 0
for i in range(0, 508438, 500):
    l.append((k, i))
    k = i
l = l[1:]
l.append((508000, 508438))

In [69]:
test_data = pd.read_csv('test_data_normalized.csv')

In [71]:
res_test = pd.DataFrame(data={'tokens': [], 'pos': [], 'type': []})

In [99]:
for every in l:
    tok_pos_type_test = test_data[every[0]:every[1]].description.apply(lambda s: pd.Series(get_chains(s)))
    tok_pos_type_test.columns = ['tokens', 'pos', 'type']
    res_test = pd.concat([res_test, tok_pos_type_test])
    #print('Конец обработки на', every)

In [73]:
res_test.to_csv('tok_pos_type_test.csv', encoding='utf-8')

## Количество токенов

In [76]:
train_data = pd.read_csv('/Users/irene/Downloads/project/train_data_normalized.csv')

In [77]:
train_data.head()

,Unnamed: 0,description,title,tokenized_title,tokenized_description,tokenized_stemmed_title,tokenized_stemmed_description
0,0,"Кокон для сна малыша,пользовались меньше месяц...",Кокоби(кокон для сна),кокоби ( кокон для сон ),"кокон для сон малыш , пользоваться маленький м...",кокоб ( кокон для сон ),"кокон для сон малыш , пользова маленьк месяц ...."
1,1,"Стойка для одежды, под вешалки. С бутика.",Стойка для Одежды,стойка для одежда,"стойка для одежда , под вешалка . с бутик .",стойк для одежд,"стойк для одежд , под вешалк . с бутик ."
2,2,"В хорошем состоянии, домашний кинотеатр с blu ...",Philips bluray,philips bluray,"в хороший состояние , домашний кинотеатр с blu...",philips blura,"в хорош состоян , домашн кинотеатр с blu ra , ..."
3,3,Продам кресло от0-25кг,Автокресло,автокресло,продать кресло от0 - 25кг,автокресл,прода кресл от0 - 25кг
4,4,Все вопросы по телефону.,"ВАЗ 2110, 2003","ваза 2110 , 2003",весь вопрос по телефон .,"ваз 2110 , 2003",ве вопрос по телефон .


In [84]:
def length_in_tokens(text):
    text = str(text)
    num = text.split()
    return len(num)

Пример:

In [85]:
length_in_tokens('кокоби ( кокон для сон )')

6

In [86]:
train_data['length_of_title_in_tokens'] = train_data['tokenized_title'].apply(length_in_tokens)
train_data['length_of_description_in_tokens'] = train_data['tokenized_description'].apply(length_in_tokens)

In [88]:
train_data = train_data.drop(['Unnamed: 0'], axis=1)

In [89]:
train_data.head()

,description,title,tokenized_title,tokenized_description,tokenized_stemmed_title,tokenized_stemmed_description,length_of_title_in_tokens,length_of_description_in_tokens
0,"Кокон для сна малыша,пользовались меньше месяц...",Кокоби(кокон для сна),кокоби ( кокон для сон ),"кокон для сон малыш , пользоваться маленький м...",кокоб ( кокон для сон ),"кокон для сон малыш , пользова маленьк месяц ....",6,11
1,"Стойка для одежды, под вешалки. С бутика.",Стойка для Одежды,стойка для одежда,"стойка для одежда , под вешалка . с бутик .",стойк для одежд,"стойк для одежд , под вешалк . с бутик .",3,10
2,"В хорошем состоянии, домашний кинотеатр с blu ...",Philips bluray,philips bluray,"в хороший состояние , домашний кинотеатр с blu...",philips blura,"в хорош состоян , домашн кинотеатр с blu ra , ...",2,21
3,Продам кресло от0-25кг,Автокресло,автокресло,продать кресло от0 - 25кг,автокресл,прода кресл от0 - 25кг,1,5
4,Все вопросы по телефону.,"ВАЗ 2110, 2003","ваза 2110 , 2003",весь вопрос по телефон .,"ваз 2110 , 2003",ве вопрос по телефон .,4,5


In [90]:
test_data = pd.read_csv('/Users/irene/Downloads/project/test_data_normalized.csv')
test_data['length_of_title_in_tokens'] = test_data['tokenized_title'].apply(length_in_tokens)
test_data['length_of_description_in_tokens'] = test_data['tokenized_description'].apply(length_in_tokens)
test_data = test_data.drop(['Unnamed: 0'], axis=1)

In [91]:
train_data.to_csv('train_data_normalized_length.csv', encoding='utf-8')
test_data.to_csv('test_data_normalized_length.csv', encoding='utf-8')